# 7. Data Preprocessing with NumPy
## 9. A Loan Data Practical Example with NumPy
- Summary

[### 0. Import Libraries, set_printoptions, and def funct](#0-import-libraries-set_printoptions-and-def-funct)

### 0. Import Libraries, set_printoptions, and def funct

In [1]:
import numpy as np
np.__version__
np.set_printoptions(suppress=True, linewidth=100, precision=2)

In [2]:
# Function show_attr

def show_attr(arrnm: str) -> str:
    strout = f' {arrnm}: '

    for attr in ('shape', 'ndim', 'size', 'dtype'):     #, 'itemsize'):
            arrnm_attr = arrnm + '.' + attr
            strout += f'| {attr}: {eval(arrnm_attr)} '

    return strout

### 1. Setting Up: Introduction to the Practical Example

#### What working as a data analyst in a data science team looks like
1. Introduce the project.
2. All the Tasks and Responsibilities that the head of data analytics assigned to us.
3. Examine the dataset (before we start cleaning and prepro)
4. Cleaning and Preprocessing the dataset.
5. Save it to an external *.csv file (once the data is ready to be analyzed).
6. Pass it on to the data scientists.
> DScientist: will use the cleaning and prepro dtset to construct a CRM (credit risk model), that measures probability of default.

#### Structure of the working process
- Gathering (Recopilación de información), Cleaning and Preprocessing the Data <- Data Analysts
- DAnalysts hand them over to the DScientists (ML knowlegde) to construct complex Predictive Models.
##### DAnalysts Rol:
1. Our goal is to obtain a clean and preprocessed dataset
2. We'll note down all the changes we're making to the original dataset in a documentation file where we describe what each column of the new dtset represents.
3. This info will be invaluable to the DScientists who will work with this data after us.

#### A day in the life of a DAnalyst
- Explain our role in the project.
- Examine the data.
- Import the data.
- Split the data.

#### The Case
- Rol: DAnalyst in a data science team of central bank in Europe.
- Team assignment: create a CRM which estimates the probability of default for every personal account.
- Terms like Probability of default, Recovery rate, and Credit Risk Modeling.
- Chore: Take the raw dataset and prepare it for the models the plan to run.
- Details provided:
    1. What data is stored in every column.
    2. Set of rules on how to clean and pre-process the values in each column col.
> The essence of the DAnalyst job and is much more demanding and sizable than it might initially sound.

#### Step by Step approach to the problem
1. Loan data is a sample from a larger dtset that belongs to an affiliate bank based in USA. Therefore all the values are in dollars, so we need to provide their Euro equivalents.
2. Every categorical variable must be quantified. We nee to change any text columns into numbers based on the info they contain.
    - Issue date (fecha de emisión) on each loan: transformation is straightforward since we can split the accounts by months.
    - For other cols, we only care if they provide positive or negative connotations. So we'll be turning them into __*dummy variables*__ that hold either zero or one.
3. Missing Data:
    - Furthermore when we're measuring creditworthiness we need to be extremely risk-averse and distrustful of any unavailable data.
    - That's why the consensus in the field is that missing info suggest foul play because loan applications are self reported. To elaborate since candidates fill out their loan applications manually, there is an incentive to withhold info which can lower their chances of getting a loan.
    - Of course we prefer to give out loans to applicants who can repay them. So __*if the information isn´t available, we'll just assume the worst*__.
    - What is worst varies from one column to the next, so the team has provided us with casting directions for each variable in the dtset.
    - Therefore as we go through the dtset we'll usually know whether we want to use the minimum, maximum, or some other value when taking care of missing data

> Loan info is store in a .csv file called loan-data.csv

### 2. Setting Up: Importing the Data Set

In [3]:
raw_data_np = np.genfromtxt('9_loan-data.csv',
                            delimiter=';',
                            skip_header=1,
                            autostrip=True)
display(raw_data_np)
display(show_attr('raw_data_np'))

# The entire 1st row is NAN so the skip_header=1

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

' raw_data_np: | shape: (10000, 14) | ndim: 2 | size: 140000 | dtype: float64 '

### 3. Setting Up: Checking for Incomplete Data

In [4]:
print('Number of NANs: ', np.isnan(raw_data_np).sum())

# Calc tmp_fill and orig_mean for e/col, then orig_stats e/col
print('Original Maximum: ', np.nanmax(raw_data_np))
display(tmp_fill := np.nanmax(raw_data_np) + 1)
display(orig_means := np.nanmean(raw_data_np, axis=0))

orig_stats = np.array([np.nanmin(raw_data_np, axis=0),
                       orig_means,
                       np.nanmax(raw_data_np, axis=0)])
display(orig_stats)

# orig_means =>  can see 8 cols full of NAN (warnings) 

Number of NANs:  88005
Original Maximum:  68616519.0


68616520.0

C:\Users\jmonti\AppData\Local\Temp\ipykernel_5612\1101693491.py:6: RuntimeWarning: Mean of empty slice
  display(orig_means := np.nanmean(raw_data_np, axis=0))


array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

C:\Users\jmonti\AppData\Local\Temp\ipykernel_5612\1101693491.py:8: RuntimeWarning: All-NaN slice encountered
  orig_stats = np.array([np.nanmin(raw_data_np, axis=0),
C:\Users\jmonti\AppData\Local\Temp\ipykernel_5612\1101693491.py:10: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data_np, axis=0)])


array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

### 4. Setting Up: Splitting the Dataset

In [5]:
# Get the cols_str(ixs), and the cols_num(ixs)
cols_str = np.argwhere(np.isnan(orig_means)).squeeze()
display(cols_str)

# cols_num = np.argwhere(~np.isnan(orig_means)).squeeze()
cols_num = np.argwhere(np.isnan(orig_means) == False).squeeze()
display(cols_num)

# Check: num_cols_str + num_cols_num = raw_data.shape[1]
len(cols_str) + len(cols_num) == raw_data_np.shape[1]

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

True

In [6]:
# Reload strings cols to build the str sub-array (now data)
data_str = np.genfromtxt('9_loan-data.csv',
                         delimiter=';',
                         dtype=str,
                         usecols=cols_str,
                         skip_header=1,
                         autostrip=True)
display(data_str)
show_attr('data_str')

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

' data_str: | shape: (10000, 8) | ndim: 2 | size: 80000 | dtype: <U69 '

In [7]:
# Reload numeric cols to build the num sub-array (now data)
# ALSO fill NANs w/temp_fill
data_num = np.genfromtxt('9_loan-data.csv',
                         delimiter=';',
                         usecols=cols_num,
                         skip_header=1,
                         autostrip=True,
                         filling_values=tmp_fill)
display(data_num)
display(show_attr('data_num'))
print("Num of NANs:", np.isnan(data_num).sum())

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

' data_num: | shape: (10000, 6) | ndim: 2 | size: 60000 | dtype: float64 '

Num of NANs: 0


In [8]:
# Load header strings cols to build the str sub-array (now header)
header_str = np.genfromtxt('9_loan-data.csv',
                            delimiter=';',
                            dtype=str,
                            usecols=cols_str,
                            skip_footer=raw_data_np.shape[0],
                            autostrip=True)
display(header_str)
show_attr('header_str')

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

' header_str: | shape: (8,) | ndim: 1 | size: 8 | dtype: <U19 '

In [9]:
# Load header numeric cols to build the num sub-array (now header)
header_num = np.genfromtxt('9_loan-data.csv',
                            delimiter=';',
                            dtype=str,
                            usecols=cols_num,
                            skip_footer=raw_data_np.shape[0],
                            autostrip=True)
display(header_num)
show_attr('header_num')

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U11')

' header_num: | shape: (6,) | ndim: 1 | size: 6 | dtype: <U11 '

### 5. Setting Up: Creating Checkpoints

In [10]:
# Function chkpt (checkpoint)

def chkpt(filenm: str, chk_header: np.ndarray,
          chk_data: np.ndarray) -> np.lib.npyio.NpzFile:
    np.savez(filenm, header=chk_header, data=chk_data)
    chkpt_var = np.load(f'{filenm}.npz')
    return chkpt_var

In [11]:
chkpt_tst = chkpt('chkpt-tst', header_num, data_num)
display(chkpt_tst.files)
np.array_equal(chkpt_tst['data'], data_num)

['header', 'data']

True

### 6. Manipulating Text Data: Issue Date

- Chg header of col issue_d to issue_date (more descriptive)
- Eliminate -15 (all 2015) and change month str by num (1-12) [0-for '']

In [12]:
# Chg header of col issue_d to issue_date (more descriptive)
display(header_str)
display(np.argwhere(header_str == 'issue_d'))
header_str[0] = 'issue_date'
display(f'{header_str[0] = }')
display(header_str)


array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array([[0]], dtype=int64)

"header_str[0] = 'issue_date'"

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [13]:
# Check the column AND jm- view sort in order of counts
id_uniq, id_ucnt = np.unique(data_str[:,0], return_counts=True)
display(id_uniq, id_ucnt)
id_ucnt_ixsorted = np.argsort(id_ucnt)
id_uniq[id_ucnt_ixsorted], id_ucnt[id_ucnt_ixsorted]

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

array([ 500,  757,  846,  997,  530,  770, 1061,  654,  559,  741,  847, 1095,  643], dtype=int64)

(array(['', 'Feb-15', 'Mar-15', 'Sep-15', 'Jun-15', 'May-15', 'Apr-15', 'Jan-15', 'Aug-15',
        'Nov-15', 'Dec-15', 'Jul-15', 'Oct-15'], dtype='<U69'),
 array([ 500,  530,  559,  643,  654,  741,  757,  770,  846,  847,  997, 1061, 1095], dtype=int64))

In [14]:
# Eliminate -15 
data_str[:,0] = np.chararray.strip(data_str[:,0], '-15')
display(np.unique(data_str[:,0]))

# Replace month str by number - '' = 0
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'])

for mth_num in range(13):
    data_str[:,0] = np.where(data_str[:,0] == months[mth_num],
                             mth_num,
                             data_str[:,0])
    
display(np.unique(data_str[:,0]))
display(np.unique(data_str[:,0])[id_ucnt_ixsorted])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

array(['0', '12', '5', '9', '4', '6', '1', '2', '10', '7', '11', '3', '8'], dtype='<U69')

### 7. Manipulating Text Data: Loan Status and Term

- 'loan_status' -> dummy variable 0 or 1
- 'term' -> only numeric and change name to 'term_month'

In [15]:
# 'loan_status' -> dummy var with 'loan_satus' values
display(header_str)
display(np.argwhere(header_str == 'loan_status'))
display(np.unique(data_str[:,1]))

bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])
# good = ['Current', 'Fully Paid', 'In Grace Period', 'Issued', 'Late (16-30 days)']
data_str[:,1] = np.where(np.isin(data_str[:,1], bad), 0, 1)

display(np.unique(data_str[:,1]))

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array([[1]], dtype=int64)

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

array(['0', '1'], dtype='<U69')

In [16]:
# 'term'
display(header_str)
display(np.argwhere(header_str == 'term'))
display(np.unique(data_str[:,2]))

# Eliminate letters
data_str[:,2] = np.chararray.strip(data_str[:,2], ' months')
display(np.unique(data_str[:,2]))

# replace '' by 60 (worst case)
data_str[:,2] = np.where(data_str[:,2] == '',
                         '60',
                         data_str[:,2])

np.unique(data_str[:,2])

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array([[2]], dtype=int64)

array(['', '36 months', '60 months'], dtype='<U69')

array(['', '36', '60'], dtype='<U69')

array(['36', '60'], dtype='<U69')

In [17]:
# Change header_str 'term' by 'term_month'
header_str[2] = 'term_month'

### 8. Manipulating Text Data: Grade and Sub Grade

- 'sub_grade' includes grade. But use Grade to fill the empties sub_grades with the worst case (5). Remaining sub_grade empties fill with new worst category (H1)
- Remove grade from data and header

In [18]:
# Let´s see grade and sub_grade columns
display(header_str)
display(np.argwhere(header_str == 'grade'))
display(np.unique(data_str[:,3]))

display(np.argwhere(header_str == 'sub_grade'))
display(np.unique(data_str[:,4], return_counts=True))


array(['issue_date', 'loan_status', 'term_month', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

array([[3]], dtype=int64)

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

array([[4]], dtype=int64)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([514, 285, 278, 239, 323, 502, 509, 517, 530, 553, 494, 629, 567, 586, 564, 423, 391, 267,
        250, 255, 223, 235, 162, 171, 139, 114,  94,  52,  34,  43,  16,  19,  10,   3,   7,   2],
       dtype=int64))

In [19]:
# Replace de empties subgrades w/corresponding grade + 5 (worst case)
for gd in np.unique(data_str[:,3])[1:]:
    data_str[:,4] = np.where((data_str[:,4] == '') & (data_str[:,3] == gd),
                             gd + '5',
                             data_str[:,4])
    
display(np.unique(data_str[:,4], return_counts=True))



(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5],
       dtype=int64))

In [20]:
# Replace the remaining sub_grade '' w/ H1
data_str[:,4] = np.where(data_str[:,4] == '', 'H1', data_str[:,4])
display(np.unique(data_str[:,4], return_counts=True))

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
        'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
        'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69'),
 array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250,
        255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5,   9],
       dtype=int64))

In [21]:
# Remove grade col from header
header_str = np.delete(header_str, 3)
display(header_str)


array(['issue_date', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [22]:
# Remove grade from data
data_str = np.delete(data_str, 3, axis=1)
np.unique(data_str[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [23]:
# Convert all the sub_grades to num (A1-1 and H1...)
display(sze := np.unique(data_str[:,3]).size)   # or .shap[0]
dic_sub_gd= dict(zip(np.unique(data_str[:,3]), range(1, sze + 1)))
# print(dic)

for k in np.unique(data_str[:,3]):
    data_str[:,3] = np.where(data_str[:,3] == k,
                             dic_sub_gd[k],
                             data_str[:,3])
    

np.unique(data_str[:,3])

36

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

### 9. Manipulating Text Data: Verification Status & URL

- 'verification_status' -> dummy (0-1)
- 'url' -> check that's not necessary

In [24]:
# Let's see them
display(header_str)
display(np.unique(data_str[:,4], return_counts=True))
display(np.unique(data_str[:,5], return_counts=True))

array(['issue_date', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

(array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69'),
 array([ 500, 2673, 4116, 2711], dtype=int64))

(array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=12606806',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=13026045',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1312426', ...,
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8138291',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8214572',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=849994'], dtype='<U69'),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [25]:
# Make verification_status as dummy (0-bad)
bad = np.array(['', 'Not Verified'])
# good = ('Source Verified', 'Verified')
data_str[:,4] = np.where(np.isin(data_str[:,4], bad), 0, 1)
display(np.unique(data_str[:,4], return_counts=True))

(array(['0', '1'], dtype='<U69'), array([3173, 6827], dtype=int64))

### 10. Manipulating Text Data: State Address

- 'addr_state': One state missing and lot of ''

In [26]:
# 1. Display status
display(header_str)
uniq_states = np.unique(data_str[:,6], return_counts=True)
display(uniq_states)
np.sort(-uniq_states[1])

array(['issue_date', 'loan_status', 'term_month', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27], dtype=int64))

array([-1336,  -777,  -758,  -690,  -500,  -389,  -341,  -321,  -320,  -312,  -267,  -261,  -242,
        -222,  -220,  -216,  -210,  -201,  -160,  -156,  -152,  -148,  -143,  -143,  -130,  -119,
        -116,  -108,  -107,   -84,   -84,   -83,   -74,   -74,   -61,   -58,   -57,   -49,   -44,
         -40,   -28,   -27,   -27,   -27,   -26,   -25,   -24,   -17,   -16,   -10], dtype=int64)

In [27]:
# 2. Sort in descending order the states - np.argsort()
ixs = np.argsort(-uniq_states[1])
display(uniq_states[0][ixs])
display(uniq_states[1][ixs])

array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
       'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
       'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
       'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69')

array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
        216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
         84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
         25,   24,   17,   16,   10], dtype=int64)

In [31]:
# 3. Assign '' = 0, to no enter any category
data_str[:,6][data_str[:,6] == ''] = '0'

data_str[:,6][data_str[:,6] == '0']
# data_str[:,6]

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0'

### 11. Manipulating Text Data: Converting Strings and Creating a Checkpoint

### 12. Manipulating Numeric Data: Substitute Filler Values